In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3) # 26
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout2d(0.05)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3) # 24
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(32)
        self.dropout2 = nn.Dropout2d(0.05)

        self.point_conv1 = nn.Conv2d(32, 16, kernel_size=1) # 24

        self.pool1 = nn.MaxPool2d(2, 2) # 12

        self.conv3 = nn.Conv2d(16, 32, kernel_size=3) # 10
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        self.dropout3 = nn.Dropout2d(0.05)

        self.point_conv2 = nn.Conv2d(32, 16, kernel_size=1) # 10

        self.conv4 = nn.Conv2d(16, 32, kernel_size=3) # 8
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(32)
        self.dropout4 = nn.Dropout2d(0.05)

        self.point_conv3 = nn.Conv2d(32, 8, kernel_size=1) # 10

        self.conv5 = nn.Conv2d(8, 16, kernel_size=3) # 8
        self.relu5 = nn.ReLU()
        self.bn5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout2d(0.05)

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(16, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)
        x = self.point_conv1(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.bn3(x)
        x = self.dropout3(x)
        x = self.point_conv2(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.bn4(x)
        x = self.dropout4(x)
        x = self.point_conv3(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.bn5(x)
        x = self.dropout5(x)      
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return F.log_softmax(x)


In [13]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 16, 24, 24]             528
        MaxPool2d-10           [-1, 16, 12, 12]               0
           Conv2d-11           [-1, 32, 10, 10]           4,640
             ReLU-12           [-1, 32, 10, 10]               0
    

<ipython-input-12-f3854f87232f>:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [14]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [15]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 21):
    print("Epoch:", {epoch})
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch: {1}


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-7-f3854f87232f>:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.08927491307258606 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.36it/s]



Test set: Average loss: 0.0898, Accuracy: 9729/10000 (97%)

Epoch: {2}


loss=0.027091404423117638 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.75it/s]



Test set: Average loss: 0.0395, Accuracy: 9878/10000 (99%)

Epoch: {3}


...


Test set: Average loss: 0.0226, Accuracy: 9938/10000 (99%)

Epoch: {19}


loss=0.002129149856045842 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.53it/s]



Test set: Average loss: 0.0274, Accuracy: 9920/10000 (99%)

Epoch: {20}


loss=0.0028877321165055037 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.22it/s]



Test set: Average loss: 0.0208, Accuracy: 9933/10000 (99%)

